In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load in data
atp_ranks = pd.read_csv("data/atp_ranks.csv")
wta_ranks = pd.read_csv("data/wta_ranks.csv")
womens_stats = pd.read_csv("data/womens_df_prod.csv")
mens_stats = pd.read_csv("data/mens_df_prod.csv")

Merge rankings data

In [3]:
# Tied rankings
atp_ranks[~atp_ranks['rank'].str.isnumeric()]

,player,rank,week
903,Pablo Cuevas,904T,2023-06-26
904,Adam Neff,904T,2023-06-26
923,Menelaos Efstathiou,924T,2023-06-26
924,John McNally,924T,2023-06-26
1001,Lorenzo Gagliardo,1002T,2023-06-26
...,...,...,...
8143,Youssef Labbene,2057T,2023-08-22
8144,Ignacio Benzal Alia,2061T,2023-08-22
8145,Akram El Sallaly,2061T,2023-08-22
8146,Abhinav Sanjeev Shanmugam,2061T,2023-08-22


In [4]:
# Remove T from the rank
atp_ranks['rank_cleaned'] = pd.to_numeric(atp_ranks['rank'].str.replace('T', ''), errors='coerce')

In [5]:
atp_ranks['player_cleaned'] = atp_ranks['player'].apply(lambda word: " ".join(word.split()[1:])+", "+word.split()[0])
atp_ranks

,player,rank,week,rank_cleaned,player_cleaned
0,Carlos Alcaraz,1,2023-06-26,1,"Alcaraz, Carlos"
1,Novak Djokovic,2,2023-06-26,2,"Djokovic, Novak"
2,Daniil Medvedev,3,2023-06-26,3,"Medvedev, Daniil"
3,Casper Ruud,4,2023-06-26,4,"Ruud, Casper"
4,Stefanos Tsitsipas,5,2023-06-26,5,"Tsitsipas, Stefanos"
...,...,...,...,...,...
8145,Akram El Sallaly,2061T,2023-08-22,2061,"El Sallaly, Akram"
8146,Abhinav Sanjeev Shanmugam,2061T,2023-08-22,2061,"Sanjeev Shanmugam, Abhinav"
8147,Ioannis Xilas,2061T,2023-08-22,2061,"Xilas, Ioannis"
8148,Jesse Delaney,2065,2023-08-22,2065,"Delaney, Jesse"


In [6]:
wta_ranks

,player,rank,week
0,Swiatek Iga,1.0,2023-06-26
1,Sabalenka Aryna,2.0,2023-06-26
2,Rybakina Elena,3.0,2023-06-26
3,Pegula Jessica,4.0,2023-06-26
4,Garcia Caroline,5.0,2023-06-26
...,...,...,...
5511,Faltinger Ines,1390.0,2023-08-22
5512,Goldsmith Weinreich Paloma,1390.0,2023-08-22
5513,Ionescu Iulia Andreea,1390.0,2023-08-22
5514,Joint Maya,1390.0,2023-08-22


In [7]:
# Move the first name to the front
# This doesn't accont for people with two first names but I think those may be edge cases I don't need to worry about for now
def move_first(names):
    names = names.split()
    ret = names[0:-1]
    ret.insert(0, names[-1])
    return " ".join(ret)


wta_ranks['player_cleaned'] = wta_ranks['player'].apply(move_first)
wta_ranks['player_cleaned'] = wta_ranks['player_cleaned'].apply(lambda word: " ".join(word.split()[1:])+", "+word.split()[0])
wta_ranks


,player,rank,week,player_cleaned
0,Swiatek Iga,1.0,2023-06-26,"Swiatek, Iga"
1,Sabalenka Aryna,2.0,2023-06-26,"Sabalenka, Aryna"
2,Rybakina Elena,3.0,2023-06-26,"Rybakina, Elena"
3,Pegula Jessica,4.0,2023-06-26,"Pegula, Jessica"
4,Garcia Caroline,5.0,2023-06-26,"Garcia, Caroline"
...,...,...,...,...
5511,Faltinger Ines,1390.0,2023-08-22,"Faltinger, Ines"
5512,Goldsmith Weinreich Paloma,1390.0,2023-08-22,"Goldsmith Weinreich, Paloma"
5513,Ionescu Iulia Andreea,1390.0,2023-08-22,"Ionescu Iulia, Andreea"
5514,Joint Maya,1390.0,2023-08-22,"Joint, Maya"


In [8]:
atp_ranks.columns

Index(['player', 'rank', 'week', 'rank_cleaned', 'player_cleaned'], dtype='object')

In [9]:
wta_ranks = wta_ranks.drop(columns = ["player"])
atp_ranks = atp_ranks.drop(columns = ['player','rank'])
atp_ranks = atp_ranks.rename(columns ={"rank_cleaned":"rank"})

In [10]:
ranks = pd.concat([atp_ranks, wta_ranks])
ranks = ranks.rename(columns = {'week':'date','player_cleaned':'player'})
ranks

,date,rank,player
0,2023-06-26,1.0,"Alcaraz, Carlos"
1,2023-06-26,2.0,"Djokovic, Novak"
2,2023-06-26,3.0,"Medvedev, Daniil"
3,2023-06-26,4.0,"Ruud, Casper"
4,2023-06-26,5.0,"Tsitsipas, Stefanos"
...,...,...,...
5511,2023-08-22,1390.0,"Faltinger, Ines"
5512,2023-08-22,1390.0,"Goldsmith Weinreich, Paloma"
5513,2023-08-22,1390.0,"Ionescu Iulia, Andreea"
5514,2023-08-22,1390.0,"Joint, Maya"


In [11]:
ranks['player'] = ranks["player"].apply(lambda x: x.title())
ranks.to_csv("data/ranks.csv", index=False)

In [12]:
stats = pd.concat([womens_stats, mens_stats])
stats[["name_home","name_away"]] = stats[["name_home","name_away"]].apply(lambda column: column.apply(lambda x: x.title()))
stats

,event_id,player_id_home,name_home,seed_home,aces_home,backhand_errors_home,backhand_unforced_errors_home,backhand_winners_home,breakpoints_won_home,double_faults_home,...,games_won,max_games_in_a_row,avg_game_length_set4,set4_length,avg_game_length_set5,set5_length,set4_diff,set4_games,set5_diff,set5_games
0,sr:sport_event:42023445,sr:competitor:66968,"Kudermetova, Veronika",12.0,14,13,8,9,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sr:sport_event:42023447,sr:competitor:41355,"Bogdan, Ana",NaN,6,12,7,7,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sr:sport_event:42023449,sr:competitor:256547,"Parks, Alycia",NaN,5,8,17,3,4,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sr:sport_event:42023459,sr:competitor:99131,"Bucsa, Cristina",NaN,6,34,15,10,6,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sr:sport_event:42023461,sr:competitor:44834,"Pegula, Jessica",4.0,3,20,14,11,5,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,sr:sport_event:43216857,sr:competitor:163504,"Medvedev, Daniil",3.0,8,10,11,7,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
491,sr:sport_event:43216863,sr:competitor:407573,"Alcaraz, Carlos",1.0,3,5,10,12,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
492,sr:sport_event:43216849,sr:competitor:808628,"Shelton, Ben",NaN,5,20,11,4,2,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
493,sr:sport_event:43216855,sr:competitor:407573,"Alcaraz, Carlos",1.0,0,25,14,12,1,0,...,NaN,NaN,6.661111,59.95,NaN,NaN,-3.0,9.0,NaN,NaN


In [13]:
stats_c = stats[["date","name_home", "seed_home", "name_away", "seed_away","men"]]
stats_c

,date,name_home,seed_home,name_away,seed_away,men
0,2023-06-26,"Kudermetova, Veronika",12.0,"Kanepi, Kaia",NaN,False
1,2023-06-26,"Bogdan, Ana",NaN,"Samsonova, Liudmila",15.0,False
2,2023-06-26,"Parks, Alycia",NaN,"Friedsam, Anna-Lena",NaN,False
3,2023-06-26,"Bucsa, Cristina",NaN,"Rakhimova, Kamilla",NaN,False
4,2023-06-26,"Pegula, Jessica",4.0,"Davis, Lauren",NaN,False
...,...,...,...,...,...,...
490,2023-08-22,"Medvedev, Daniil",3.0,"Rublev, Andrey",8.0,True
491,2023-08-22,"Alcaraz, Carlos",1.0,"Zverev, Alexander",12.0,True
492,2023-08-22,"Shelton, Ben",NaN,"Djokovic, Novak",2.0,True
493,2023-08-22,"Alcaraz, Carlos",1.0,"Medvedev, Daniil",3.0,True


In [14]:
print("stats_c")

for col in stats_c.columns:
    if any(stats_c[col].isna()):
        print(col)

print("ranks")
for col in ranks.columns:
    if any(ranks[col].isna()):
        print(col)

stats_c
seed_home
seed_away
ranks


Merging ranks and stats

In [15]:
import regex as re
from functools import reduce

In [16]:
# merge stats and ranks (only for home)
merged = stats.merge(ranks, how = "left", left_on=["date","name_home"], right_on=["date","player"]).rename(columns={"rank":"rank_home","player":"player_home"})

In [17]:
# Find missing players for home
missing_players = []
found_players = []
to_fix = []
for row in merged[(merged['rank_home'].isna())].iterrows():
    name = row[1].name_home
    name_list = re.split(", | ",name)
    potential_ppl = []
    for part in name_list:
        exists = ranks['player'].apply(lambda x: bool(re.search(part, x, flags=re.IGNORECASE)))
        potential_ppl.append(ranks[exists])
    
    found = reduce(lambda x, y: pd.merge(x, y, on=['date','rank','player']), potential_ppl)
    if found.empty:
        # Try again only search for last name
        exists = ranks['player'].apply(lambda x: bool(re.search(name_list[0], x, flags=re.IGNORECASE)))
        found = ranks[exists]
        if len(found)==4:
            if name not in found_players:
                print(f"FOUND: {name}")
                corrected_values = found.copy()
                corrected_values.loc[:, 'player_corrected'] = name
                to_fix.append(ranks.merge(corrected_values, on=["date", 'rank','player']))
                found_players.append(name)
        else:
            if name not in missing_players:
                print(f"LOST: {name}")
                missing_players.append(name)
        
    else:
        if name not in found_players:
            print(f"FOUND: {name}")
            corrected_values = found.copy()
            corrected_values.loc[:,'player_corrected'] = name
            to_fix.append(ranks.merge(corrected_values, on=["date", 'rank','player']))
            found_players.append(name)

# # represents a new dataframe of the remaining ranked players that were missed in the previous iteration
missed_ranks = pd.concat(to_fix)
missed_ranks = missed_ranks.drop_duplicates()

merged2 = merged.merge(missed_ranks, how = "left", left_on=["date","name_home"], right_on=["date","player_corrected"])#.drop(columns=["player_x","player_y"])
merged2['name_home'] = merged2.apply(lambda row: row['name_home'] if pd.isna(row['player_corrected']) else row['player_corrected'], axis=1)
merged2['rank_home'] = merged2.apply(lambda row: row['rank_home'] if pd.isna(row['rank']) else row['rank'], axis=1)

FOUND: Yuan, Yue
FOUND: Tsurenko, Lesia
FOUND: Burrage, Jodie Anna
FOUND: Kasatkina, Daria
FOUND: Cirstea, Sorana
LOST: Cristian, Jaqueline Adina
FOUND: Stephens, Sloane
FOUND: Minnen, Greet
LOST: Riske-Amritraj, Alison
FOUND: Begu, Irina-Camelia
LOST: Fernandez, Leylah Annie
FOUND: Andreescu, Bianca
FOUND: Azarenka, Victoria
FOUND: Parrizas Diaz, Nuria
FOUND: Samsonova, Liudmila
LOST: Maria, Tatjana
FOUND: Schmiedlova, Anna-Karolina
FOUND: Krunic, Aleksandra
FOUND: Gauff, Coco
FOUND: Miyazaki, Yuriko Lily
FOUND: Tig, Patricia Maria
LOST: Wolf, Jeffrey John
FOUND: Etcheverry, Tomas Martin
FOUND: Galan, Daniel Elahi
FOUND: Barrios Vera, Marcelo Tomas
FOUND: Tirante, Thiago Agustin
FOUND: Olivieri, Genaro Alberto
FOUND: Varillas, Juan Pablo
FOUND: Cerundolo, Juan Manuel
FOUND: Hsu, Yu Hsiou


In [18]:
merged2 = merged2.drop(columns = ["rank","player","player_corrected","player_home"])
merged2

,event_id,player_id_home,name_home,seed_home,aces_home,backhand_errors_home,backhand_unforced_errors_home,backhand_winners_home,breakpoints_won_home,double_faults_home,...,max_games_in_a_row,avg_game_length_set4,set4_length,avg_game_length_set5,set5_length,set4_diff,set4_games,set5_diff,set5_games,rank_home
0,sr:sport_event:42023445,sr:competitor:66968,"Kudermetova, Veronika",12.0,14,13,8,9,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0
1,sr:sport_event:42023447,sr:competitor:41355,"Bogdan, Ana",NaN,6,12,7,7,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.0
2,sr:sport_event:42023449,sr:competitor:256547,"Parks, Alycia",NaN,5,8,17,3,4,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.0
3,sr:sport_event:42023459,sr:competitor:99131,"Bucsa, Cristina",NaN,6,34,15,10,6,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.0
4,sr:sport_event:42023461,sr:competitor:44834,"Pegula, Jessica",4.0,3,20,14,11,5,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,sr:sport_event:43216857,sr:competitor:163504,"Medvedev, Daniil",3.0,8,10,11,7,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
984,sr:sport_event:43216863,sr:competitor:407573,"Alcaraz, Carlos",1.0,3,5,10,12,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
985,sr:sport_event:43216849,sr:competitor:808628,"Shelton, Ben",NaN,5,20,11,4,2,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.0
986,sr:sport_event:43216855,sr:competitor:407573,"Alcaraz, Carlos",1.0,0,25,14,12,1,0,...,NaN,6.661111,59.95,NaN,NaN,-3.0,9.0,NaN,NaN,1.0


In [19]:
# # Do the same thing for away
merged = merged2.merge(ranks, how = "left", left_on=["date","name_away"], right_on=["date","player"]).rename(columns={"rank":"rank_away","player":"player_away"})

# Find missing players for home
found_players = []
to_fix = []
for row in merged[(merged['rank_away'].isna())].iterrows():
    name = row[1].name_away
    name_list = re.split(", | ",name)
    potential_ppl = []
    for part in name_list:
        exists = ranks['player'].apply(lambda x: bool(re.search(part, x, flags=re.IGNORECASE)))
        potential_ppl.append(ranks[exists])
    
    found = reduce(lambda x, y: pd.merge(x, y, on=['date','rank','player']), potential_ppl)
    if found.empty:
        # Try again only search for last name
        exists = ranks['player'].apply(lambda x: bool(re.search(name_list[0], x, flags=re.IGNORECASE)))
        found = ranks[exists]
        if len(found)==4:
            if name not in found_players:
                print(f"FOUND: {name}")
                corrected_values = found.copy()
                corrected_values.loc[:, 'player_corrected'] = name
                to_fix.append(ranks.merge(corrected_values, on=["date", 'rank','player']))
                found_players.append(name)
        else:
            if name not in missing_players:
                print(f"LOST: {name}")
                missing_players.append(name)
        
    else:
        if name not in found_players:
            print(f"FOUND: {name}")
            corrected_values = found.copy()
            corrected_values.loc[:,'player_corrected'] = name
            to_fix.append(ranks.merge(corrected_values, on=["date", 'rank','player']))
            found_players.append(name)

missed_ranks = pd.concat(to_fix)
missed_ranks = missed_ranks.drop_duplicates()

merged2 = merged.merge(missed_ranks, how = "left", left_on=["date","name_away"], right_on=["date","player_corrected"])
merged2['name_away'] = merged2.apply(lambda row: row['name_away'] if pd.isna(row['player_corrected']) else row['player_corrected'], axis=1)
merged2['rank_away'] = merged2.apply(lambda row: row['rank_away'] if pd.isna(row['rank']) else row['rank'], axis=1)

FOUND: Samsonova, Liudmila
FOUND: Azarenka, Victoria
FOUND: Teichmann, Jil
LOST: Mcnally, Catherine
FOUND: Gauff, Coco
FOUND: Burrage, Jodie Anna
FOUND: Schmiedlova, Anna-Karolina
FOUND: Parrizas Diaz, Nuria
FOUND: Tsurenko, Lesia
FOUND: Andreescu, Bianca
FOUND: Cirstea, Sorana
FOUND: Begu, Irina-Camelia
FOUND: Kasatkina, Daria
FOUND: Stephens, Sloane
FOUND: Starodubtseva, Yuliia
FOUND: Osaka, Naomi
FOUND: Kerber, Angelique
FOUND: Yuan, Yue
FOUND: Ruse, Elena-Gabriela
FOUND: Minnen, Greet
FOUND: Varillas, Juan Pablo
FOUND: Barrios Vera, Marcelo Tomas
FOUND: Galan, Daniel Elahi
FOUND: Cerundolo, Juan Manuel
FOUND: Tirante, Thiago Agustin
FOUND: Olivieri, Genaro Alberto
FOUND: Etcheverry, Tomas Martin
FOUND: Balazs, Attila
FOUND: Hsu, Yu Hsiou


In [20]:
# Only a few players could not be merged, we can just drop these or have a NA seed and rank
print(missing_players)

['Cristian, Jaqueline Adina', 'Riske-Amritraj, Alison', 'Fernandez, Leylah Annie', 'Maria, Tatjana', 'Wolf, Jeffrey John', 'Mcnally, Catherine']


In [21]:
merged = merged2.drop(columns = ["rank","player","player_corrected","player_away"])
merged

,event_id,player_id_home,name_home,seed_home,aces_home,backhand_errors_home,backhand_unforced_errors_home,backhand_winners_home,breakpoints_won_home,double_faults_home,...,avg_game_length_set4,set4_length,avg_game_length_set5,set5_length,set4_diff,set4_games,set5_diff,set5_games,rank_home,rank_away
0,sr:sport_event:42023445,sr:competitor:66968,"Kudermetova, Veronika",12.0,14,13,8,9,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,101.0
1,sr:sport_event:42023447,sr:competitor:41355,"Bogdan, Ana",NaN,6,12,7,7,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.0,15.0
2,sr:sport_event:42023449,sr:competitor:256547,"Parks, Alycia",NaN,5,8,17,3,4,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.0,87.0
3,sr:sport_event:42023459,sr:competitor:99131,"Bucsa, Cristina",NaN,6,34,15,10,6,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.0,70.0
4,sr:sport_event:42023461,sr:competitor:44834,"Pegula, Jessica",4.0,3,20,14,11,5,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,sr:sport_event:43216857,sr:competitor:163504,"Medvedev, Daniil",3.0,8,10,11,7,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,8.0
984,sr:sport_event:43216863,sr:competitor:407573,"Alcaraz, Carlos",1.0,3,5,10,12,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,12.0
985,sr:sport_event:43216849,sr:competitor:808628,"Shelton, Ben",NaN,5,20,11,4,2,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.0,2.0
986,sr:sport_event:43216855,sr:competitor:407573,"Alcaraz, Carlos",1.0,0,25,14,12,1,0,...,6.661111,59.95,NaN,NaN,-3.0,9.0,NaN,NaN,1.0,3.0


In [22]:
merged['rs_home'] = merged['seed_home'].fillna(merged['rank_home'])
merged['rs_away'] = merged['seed_away'].fillna(merged['rank_away'])
merged['seed_home'] = merged['seed_home'].isna() # change to if seeded or not
merged['seed_away'] = merged['seed_away'].isna() # change to if seeded or not

In [26]:
merged.to_csv("data/merged.csv", index=False)